Redoing tutorial with a different image - 
hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits

You can download it from https://archive.stsci.edu/pub/hlsp/hudf12/ (Links to an external site.).

Repeating the process - we are going to do some object detection on the image

In [ ]:
!pip install astropy
!pip install sep

Setting up packages and plot sizes

In [ ]:
import numpy as np
import pandas as pd
import sep

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]


Opening image file and reading it in 


In [ ]:
data = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")[0].data

If you are using SEP to analyze data read from FITS files with astropy.io.fits you may see an error message such as:

ValueError: Input array with dtype '>f4' has non-native byte order.
Only native byte order arrays are supported. To change the byte
order of the array 'data', do 'data = data.byteswap().newbyteorder()'
It is usually easiest to do this byte-swap operation directly after reading the array from the FITS file. You can even perform the byte swap in-place by doing

>>> data = data.byteswap(inplace=True).newbyteorder()

In [ ]:
 data = data.byteswap(inplace=True).newbyteorder()

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('render-first-setup.png')


BACKGROUND SUBTRACTION


In [ ]:
bkg = sep.Background(data)


In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
bkg_image = bkg.back()

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('render-second-background.png')

In [ ]:
bkg_rms = bkg.rms()

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('render-third-noise.png')

In [ ]:
HERE IS WHERE THE SUBTRACTION HAPPENS


In [ ]:
data_sub = data - bkg

OBJECT DETECTION

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
len(objects)

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

plt.savefig('render-four-detectedobjects.png')

In [ ]:
objects.dtype.names

ANALYSING FLUX AND PLOTTING

In [ ]:
flux_frame = pd.DataFrame(objects['flux'])
# To see all the data summarized - ease of work instead of personally calculating

flux_frame.describe()

In [ ]:
df = pd.DataFrame(objects)

In [ ]:
# Largest outlier
df['flux'].max()

# Using in built functions and formatted string we present 
max_outlier_from_sd= (df['flux'].max() - df['flux'].mean())/ df['flux'].std()
f"the greatest outlie is {max_outlier_from_sd} standard deviations from the mean"

In [ ]:
#Plotting simple histogram of flux data

plt.hist(objects['flux'], bins = 30)
plt.xlabel('Flux value')
plt.ylabel('Occurence')
plt.title('Flux Plot Histo')
plt.savefig('fluxplot.png')



APERTURE PHOTOMETRY

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

SECTION 8 - MAKING 3 COLOR FALSE IMAGE of the UDF USING RGB->f160w, f125w, f105w. Save the image as a PNG.

A little confused - do the file values replace the RBG positions?

In [ ]:
blueIntensity = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")[0].data
greenIntensity = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits")[0].data
redIntensity = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits")[0].data

I have no idea how to proceed - from online research I would do the following - 

1) Rescale the images
2) Apply each RGB layer using - rgb[0,1,0] etc
3) Plot image